In [2]:
# Load tensorflow package
import tensorflow as tf

a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)


d = tf.multiply(a,b)
e = tf.add(c,b)
f = tf.subtract(d,e)

#creating  a Session and Running it.
#A session object is the part of the TF API that communicates between Python objects 
sess = tf.Session()
outs = sess.run(f)
sess.close()
print("outs = {}".format(outs))



outs = 5


# Constructing and Managing Graph

In [3]:
print(tf.get_default_graph())
g = tf.Graph()
p = tf.constant(5)
print(g)
print(a.graph is g) # To check for default graph
print(a.graph is tf.get_default_graph())

False
True


# The "with" statement

 The with statement is used to wrap the execution of a block with
methods defined by a context manager—an object that has the special
method functions .__enter__() to set up a block of code
and .__exit__() to exit the block. 
* Opening a session using the with clauser will ensure the session is automatically closed once all computations are done.

In [4]:
g1 = tf.get_default_graph()
g2 = tf.Graph()

print(g1 is tf.get_default_graph())

with g2.as_default():
    print(g1 is tf.get_default_graph())
    
print(g1 is tf.get_default_graph())


True
False
True


# Fetches

Recall, outs = sess.run(f)  we request one specific node (node f) by passing the
variable it was assigned to as an argument to the sess.run() method. This argument
is called fetches, corresponding to the elements of the graph we wish to compute.

In [5]:
with tf.Session() as sess:
    fetches = [a,b,c,d,e,f]
    outs =sess.run(fetches)
    
print("outs = {}".format(outs))
print(type(outs[0]))

outs = [5, 2, 3, 10, 5, 5]
<type 'numpy.int32'>


# Flowing Tensors

When we construct a node in the graph, like we did with tf.add(), we are actually
creating an operation instance. These operations do not produce actual values until
the graph is executed, but rather reference their to-be-computed result as a handle
that can be passed on—flow—to another node. These handles, which we can think of
as the edges in our graph, are referred to as Tensor objects, and this is where the
name TensorFlow originates from.

In [6]:
c = tf.constant(4.0)
print(c)

Tensor("Const_4:0", shape=(), dtype=float32)


# Data Types

The basic units of data that pass through a graph are numerical, Boolean, or string elements.
We can ecplicitly choose what data type we want to work with by specifying it when
 we create the Tensor object.

In [8]:
c = tf.constant(4.0, dtype=tf.float64)
print(c)
print(c.dtype)

Tensor("Const_5:0", shape=(), dtype=float64)
<dtype: 'float64'>


In [11]:
v = tf.constant("4.0", dtype=tf.string)
print(v)
print(v.dtype)

Tensor("Const_6:0", shape=(), dtype=string)
<dtype: 'string'>


# Casting

It is important to make sure the data types match throughout the graph - 
performing an operation with nonmatching data types will result in an exception.To change
the data type setting of a Tensor object, we can use the tf.cast() operation, passing the relevant 
Tensor and the new data type of interest as first and second arguments respectively.


In [17]:
x = tf.constant([1,2,3], name='x', dtype=tf.float32)
print(x.dtype)
x = tf.cast(x,tf.int64)
print(x.dtype)
x = tf.cast(x,tf.string)
print(x.dtype)

<dtype: 'float32'>
<dtype: 'int64'>
<dtype: 'string'>


# Tensor Arrays and Shapes

In [18]:
import tensorflow as tf
import numpy as np

c = tf.constant([[1,2,3],[4,5,6]])

print("Python List input: {}".format(c.get_shape()))

c = tf.constant(np.array
                ([
                    [[1,2,3], 
                    [4,5,6]], 
                    [[1,1,1], 
                    [2,2,2]]
                            ])) 
print("3d NumPy array input: {}".format(c.get_shape()))

Python List input: (2, 3)
3d NumPy array input: (2, 2, 3)


The get_shape() method returns the shape of the tensor as a tuple of integers. The number of integers corresponds to the 
number of dimensions of the tensor and each integer is the numter of array entries along that dimension.

In [27]:
sess = tf.InteractiveSession()
c = tf.linspace(0.0, 4.0, 5)
print("The content of 'c' : \n {} \n".format(c.eval()))
sess.close()

The content of 'c' : 
 [ 0.  1.  2.  3.  4.] 



The sequence generator tf.linspace(a,b,n) where n evenly spaced values from a to b.

tf.InteractiveSession() allows the replacement of tf.Session() without the need of assigning a variable to hold the session.

# Matrix Multiplication 

Say we have a Tensor storing a matrix A and another storing a vector x, and we wish to compute the matrix product of the
two:
    Ax = b
This can be achieved by tf.matmul(A,B)

# Names

Each Tensor object has an identifying name. This name is an intrinsic string name, not to be confused with the name of the
variable. As with dtype, we can use the .name atrribute to see the name of the object.

In [30]:
with tf.Graph().as_default():
    c1 = tf.constant(4, dtype=tf.float64, name = "c")
    c2 = tf.constant(4, dtype=tf.int32, name = "c")
    
print(c1.name)
print(c2.name)

c:0
c_1:0


Objects residing within the same graph cannot have the same name - TensorFlow forbids it. As a consequence, it will
automatically add an underscore/space and a number to distinguish the two. However, both objects have the same name 
when they are associated with different graphs.

# Name scopes 

In some cases when dealing with a large, complicated graph, we would like to create some node grouping to make it easier
to follow and manage. This is done by using tf.name_scope("prefix") together with the useful with clause again:
    

In [31]:
with tf.Graph().as_default():
    c1 = tf.constant(4, dtype=tf.float64, name = "c")
    with tf.name_scope("prefix_name"):
        c2 = tf.constant(4, dtype=tf.int32, name = "c")
        c3 = tf.constant(4, dtype=tf.float64, name = "c")
    
    print(c1.name)
    print(c2.name)
    print(c3.name)

c:0
prefix_name/c:0
prefix_name/c_1:0


In this example we have grouped objects contained in variable c2 and c3 under the scope prefix_name, which show up as 
a prefix in their names. This is useful for visualization of the graph structure.

# Variables, Placeholders, and Simple Optimization